In [ ]:
!pip install rdkit
!pip install torch torch-geometric

import torch
from torch_geometric.datasets import QM9
from torch_geometric.data import DataLoader
from torch_geometric.nn import GINConv, global_mean_pool
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
db = pd.read_csv('peptides.csv')
db.head(30)

,ID,COMPLEXITY,NAME,N TERMINUS,SEQUENCE,C TERMINUS,SYNTHESIS TYPE,TARGET GROUP,TARGET OBJECT
0,1,Multimer,Distinctin,NaN,NLVSGLIEARKYLEQLHRKLKNCKV ENREVPPGFTALIKTLR...,NaN,Ribosomal,Gram+ Gram-,Lipid Bilayer
1,3,Multimer,Halocidin,NaN,WLNALLHHGLNCAKGVLA ALLHHGLNCAKGVLA,AMD AMD,Ribosomal,Gram+,Lipid Bilayer
2,4,Multimer,Khal,NaN,KWLNALLHHGLNCAKGVLA ALLHHGLNCAKGVLA,AMD AMD,Synthetic,Gram+ Gram-,Lipid Bilayer
3,6,Multi-Peptide,Enterocin X,NaN,SNDSLWYGVGQFMGKQANCITNHPVKHMIIPGYCLSKILG IA...,NaN,Ribosomal,Gram+,Lipid Bilayer
4,7,Multi-Peptide,EAFP1 + EAFP2,NaN,XTCASRCPRPCNAGLCCSIYGYCGSGNAYCGAGNCRCQCRG X...,NaN,Ribosomal,Fungus,Lipid Bilayer
5,8,Monomer,NaN,C16,KVvvKWVvKvVK,AMD,Synthetic,NaN,NaN
6,9,Multi-Peptide,Lactococcin-Q alpha + Lactococcin-G alpha,NaN,SIWGDIGQGVGKAAYWVGKAMGNMSDVNQASRINRKKKH GTW...,NaN,Ribosomal,Gram+,Lipid Bilayer
7,10,Monomer,NaN,NaN,LFIFFF,AMD,Synthetic,NaN,NaN
8,11,Monomer,"Cathelicidin-1, CATH-1, Fowlicidin-1",NaN,RVKRVWPLVIRTVIAGYNLYRAIKKK,NaN,Ribosomal,NaN,NaN
9,12,Monomer,NaN,NaN,RKRIHIGPGRAFYTT,NaN,Synthetic,NaN,NaN


In [ ]:
# Recuperando solo los péptidos antimibrobianos de la base
aux = []
for i in range(len(db)):

    a = str(db["NAME"][i])
    if "antimicrobial" in a:
        aux.append(list(db.iloc[i,:]))

# Lo convertimos de nuevo a un archivo tipo pandas
aux = pd.DataFrame(aux)
aux

,0,1,2,3,4,5,6,7,8
0,629,Monomer,"Cathelicidin-5, myeloid antimicrobial peptide ...",NaN,GGLRSLGRKILRAWKKYG,AMD,Synthetic,NaN,NaN
1,648,Monomer,"Thrombocidin-1 antimicrobial variant, TC-1",NaN,AELRCMCIKTTSGIHPKNIQSLEVIGKGTHCNQVEVIATLKDGRKI...,NaN,Ribosomal,NaN,NaN
2,659,Monomer,"Tracheal antimicrobial peptide, TAP",NaN,NPVSCVRNKGICVPIRCPGSMKQIGTCVGRAVKCCRKK,NaN,Ribosomal,NaN,NaN
3,1071,Monomer,"Cathelin-related antimicrobial peptide, Cramp,...",NaN,GLLRKGGEKIGEKLKKIGQKIKNFFQKLVPQPEQ,NaN,Ribosomal,NaN,NaN
4,1214,Monomer,Proline-rich antimicrobial peptide 1,NaN,DIQIPGIKKPTHRDIIIPNWNPNVRTQPWQRFGGNKS,NaN,Ribosomal,NaN,NaN
...,...,...,...,...,...,...,...,...,...
84,12336,Monomer,"Cathelicidin-5, myeloid antimicrobial peptide ...",NaN,KGLRKLGRKILRAWKKGGPIIVPIIRI,NaN,Synthetic,NaN,NaN
85,13910,Monomer,"Fowlicidin, Myeloid antimicrobial peptide 27",NaN,RFGRFLRKIRRFRPKVTITIQGSARFG,AMD,Ribosomal,NaN,NaN
86,13911,Monomer,"PMAP-36 (2-17) [K7,11W]-Fowlicidin, Myeloid an...",NaN,RFRRLRWKTRWRLKKIRFGRFLRKIRRFRPK,AMD,Synthetic,NaN,NaN
87,16332,Monomer,"Cathelicidin antimicrobial peptide isoform X1,...",NaN,RRIPFWPIPLRWQWPPPWFPPSFPIPRISRKR,NaN,Synthetic,NaN,NaN


In [ ]:
#Convirtiendo secuencia a smiles

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

# Diccionario de aminoácidos a SMILES
amino_acids = {
    'A': 'CC(C(=O)O)N',  # Alanina
    'R': 'C(C[C@H](N)C(=O)O)N=C(N)N',  # Arginina
    'N': 'C(C(C(=O)O)N)C(=O)N',  # Asparagina
    'D': 'C(C(C(=O)O)N)C(=O)O',  # Ácido Aspártico
    'C': 'C(C(C(=O)O)N)S',  # Cisteína
    'Q': 'C(CC(=O)N)[C@H](N)C(=O)O',  # Glutamina
    'E': 'C(CC(=O)O)[C@H](N)C(=O)O',  # Ácido Glutámico
    'G': 'C(C(=O)O)N',  # Glicina
    'H': 'C1C(C(=O)O)CN=C1N',  # Histidina
    'I': 'CC[C@H](C)[C@H](N)C(=O)O',  # Isoleucina
    'L': 'CC(C)C[C@H](N)C(=O)O',  # Leucina
    'K': 'C(CCN)[C@H](N)C(=O)O',  # Lisina
    'M': 'CSCC[C@H](N)C(=O)O',  # Metionina
    'F': 'C1=CC=C(C=C1)[C@H](C(=O)O)N',  # Fenilalanina
    'P': 'C1CC(NC1)C(=O)O',  # Prolina
    'S': 'C(C(C(=O)O)N)O',  # Serina
    'T': 'C[C@H](O)[C@H](N)C(=O)O',  # Treonina
    'W': 'C1=CNC2=CC=CC=C21[C@H](C(=O)O)N',  # Triptófano
    'Y': 'C1=CC=C(C=C1)C(C(=O)O)N',  # Tirosina
    'V': 'CC(C)[C@H](N)C(=O)O',  # Valina
}

# Función para convertir una secuencia de aminoácidos a SMILES
def sequence_to_smiles(sequence):
    smiles = ""
    for aa in sequence:
        smiles += amino_acids.get(aa, "")  # Obtén el SMILES del aminoácido
    return smiles

In [ ]:
secuencias = db["SEQUENCE"]

In [ ]:
secuencias[1]

'LFIFFF'

In [ ]:
secuencias_smiles = []
for secuencia in secuencias:
    smiles = sequence_to_smiles(secuencia)
    secuencias_smiles.append(smiles)

In [ ]:
secuencias_smiles[1]

'CC(C)C[C@H](N)C(=O)OC1=CC=C(C=C1)[C@H](C(=O)O)NCC[C@H](C)[C@H](N)C(=O)OC1=CC=C(C=C1)[C@H](C(=O)O)NC1=CC=C(C=C1)[C@H](C(=O)O)NC1=CC=C(C=C1)[C@H](C(=O)O)N'

In [ ]:
len(secuencias_smiles)

9275

In [ ]:
!conda install -c conda-forge rdkit
!pip install torch-geometric

/bin/bash: line 1: conda: command not found


In [ ]:
#!pip install torch-geometric
#!pip install torch-scatter
#!pip install torch-sparse
#!pip install torch-cluster
#!pip install torch-spline-conv
#!pip install torch-geometric

In [ ]:
import torch
from rdkit import Chem
from rdkit.Chem import rdmolops
from torch_geometric.data import Data
import numpy as np

# Función para convertir SMILES a Data() de PyTorch Geometric
def smiles_to_data(smiles):
    # Convertir la cadena SMILES a un objeto RDKit Mol
    mol = Chem.MolFromSmiles(smiles)

    if mol is None:
        raise ValueError("Error al procesar SMILES: " + smiles)

    # Obtener la matriz de adyacencia de la molécula (enlaces entre átomos)
    adj = rdmolops.GetAdjacencyMatrix(mol)

    # Nodos: Extraer las características de los átomos
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append(atom_to_feature_vector(atom))

    # Convertir la lista de características a un tensor de características
    x = torch.tensor(np.array(atom_features), dtype=torch.float)

    # Bordes: Obtener los índices de los enlaces
    edge_index = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append([i, j])
        edge_index.append([j, i])  # Agregar ambos sentidos del enlace

    # Convertir los índices de enlaces a un tensor
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # Crear el objeto Data() de PyTorch Geometric
    data = Data(x=x, edge_index=edge_index, y=torch.tensor([0], dtype=torch.float)

    return data

# Función para obtener las características de un átomo en formato vectorial
def atom_to_feature_vector(atom):
    # Aquí puedes definir más características del átomo si lo necesitas
    return [
        atom.GetAtomicNum(),  # Número atómico
        atom.GetDegree(),     # Grado de valencia
        atom.GetFormalCharge(),  # Carga formal
        atom.GetNumRadicalElectrons(),  # Número de electrones radicales
        atom.GetHybridization().real,  # Hibridación
        atom.GetIsAromatic()  # Si es aromático
    ]

# Ejemplo de SMILES
smiles = "CCO"  # Etanol

# Convertir SMILES a Data
data = smiles_to_data(smiles)

# Imprimir las características
print(data)

Data(x=[3, 6], edge_index=[2, 4])


In [ ]:
data2 = smiles_to_data(secuencias_smiles[1])

In [ ]:
data2

Data(x=[62, 6], edge_index=[2, 130])

In [ ]:
data2.x

tensor([[6., 1., 0., 0., 4., 0.],
        [6., 3., 0., 0., 4., 0.],
        [6., 1., 0., 0., 4., 0.],
        [6., 2., 0., 0., 4., 0.],
        [6., 3., 0., 0., 4., 0.],
        [7., 1., 0., 0., 4., 0.],
        [6., 3., 0., 0., 3., 0.],
        [8., 1., 0., 0., 3., 0.],
        [8., 2., 0., 0., 3., 0.],
        [6., 3., 0., 0., 3., 1.],
        [6., 2., 0., 0., 3., 1.],
        [6., 2., 0., 0., 3., 1.],
        [6., 3., 0., 0., 3., 1.],
        [6., 2., 0., 0., 3., 1.],
        [6., 2., 0., 0., 3., 1.],
        [6., 3., 0., 0., 4., 0.],
        [6., 3., 0., 0., 3., 0.],
        [8., 1., 0., 0., 3., 0.],
        [8., 1., 0., 0., 3., 0.],
        [7., 2., 0., 0., 4., 0.],
        [6., 2., 0., 0., 4., 0.],
        [6., 2., 0., 0., 4., 0.],
        [6., 3., 0., 0., 4., 0.],
        [6., 1., 0., 0., 4., 0.],
        [6., 3., 0., 0., 4., 0.],
        [7., 1., 0., 0., 4., 0.],
        [6., 3., 0., 0., 3., 0.],
        [8., 1., 0., 0., 3., 0.],
        [8., 2., 0., 0., 3., 0.],
        [6., 3

In [ ]:
# Definir el modelo GIN
class GIN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_dim, output_dim):
        super(GIN, self).__init__()
        self.conv1 = GINConv(Sequential(Linear(num_node_features, hidden_dim), ReLU(), Linear(hidden_dim, hidden_dim)))
        self.conv2 = GINConv(Sequential(Linear(hidden_dim, hidden_dim), ReLU(), Linear(hidden_dim, hidden_dim)))
        self.fc1 = Linear(hidden_dim, hidden_dim)
        self.fc2 = Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x